In [1]:
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [2]:
from datasets import load_dataset # for loading the datasets
from transformers import pipeline # loading the pipeline

In [3]:
# function for reading the dataset
def get_dataset(datasetname=None, version=None):
    """Function to get a dataset by name and version.

    Args:
        name (str): Name of the dataset.
        version (str): Version of the dataset.

    Returns:
        dataset: Loaded dataset with the specified name and version.
        Exception: Returns the exception if loading the dataset fails.
    """
    try:
        return load_dataset(datasetname, version)
    except Exception as e:
        return e


# function to display dataset as dataframe
def display_dataframe(dataset=None, name=None, nrows=None):
    """Function to display a dataframe from a dataset.

    Args:
        dataset: Dataset object containing the dataframe.
        name (str): Name of the dataframe within the dataset.
        nrows (int): Number of rows to display from the dataframe.

    Returns:
        pandas.DataFrame: DataFrame containing the specified number of rows.
        Exception: Returns the exception if an error occurs.
    """
    try:
        if nrows is None:
            return dataset[name].to_pandas()
        else:
            return dataset[name].select(range(nrows)).to_pandas()
    except Exception as e:
        return e



In [4]:
class Pipelines:
    """Class to represent a collection of pipelines for various NLP tasks."""

    def __init__(self, task=None, model=None, min_length=None, max_length=None, truncation=None):
        """Initialize Pipelines object with specified parameters.

        Args:
            task (str): Name of the NLP task (e.g., "text-generation", "sentiment-analysis").
            model (str): Name or path of the pretrained model to use for the task.
            min_length (int): Minimum length of generated sequences (for text generation tasks).
            max_length (int): Maximum length of generated sequences (for text generation tasks).
            truncation (str): Truncation strategy for input sequences.
                - "only_first": Truncate only the first sequence in a pair of input sequences.
                - "only_second": Truncate only the second sequence in a pair of input sequences.
                - "longest_first": Truncate the longer of the two sequences in a pair of input sequences.
        """
        if task != 'text-classification':
          self.pipeline_obj = pipeline(
            task=task,
            model=model,
            min_length=min_length,
            max_length=max_length,
            truncation=truncation)
        else:
          self.pipeline_obj = pipeline(
            task=task,
            model=model,
            truncation=truncation)


In [5]:
# Define a class named LLMTask, which inherits from Pipelines
class LLMTask(Pipelines):

    # Constructor method
    def __init__(self, task=None, model=None, min_length=None, max_length=None, truncation=None):
        # Call the constructor of the parent class using super()
        super().__init__(task, model, min_length, max_length, truncation)

    # Method to extract text data from a DataFrame
    def get_text(self, df=None, feature=None):
        try:
            # Return a list comprehension iterating over the specified feature column of the DataFrame
            return [data for data in df[feature]]
        except Exception as e:
            # If an exception occurs, return the exception object
            return e

    # Method to perform an operation on document data
    def operation_on_document(self, df=None, name=None):
        try:
            # Call the pipeline_obj method of the parent class, passing the text data extracted using get_text method
            return self.pipeline_obj(self.get_text(df, name))
        except Exception as e:
            # If an exception occurs, return the exception object
            return e


In [6]:
# Define a class for text summarization
class Summarization:

  # Constructor method
  def __init__(self, dataset_name=None, version=None, split=None, nrows=None):
    # Retrieve dataset and display it using display_dataframe function
    self.df = display_dataframe(dataset=get_dataset(datasetname=dataset_name, version=version), name=split, nrows=nrows)

  # Method to summarize text
  def summarize_text(self, task="summarization", model=None, min_length=None, max_length=None, truncation=None, featurename=None):
    try:
      # Initialize a summarizer object using LLMTask class
      summarizer = LLMTask(task=task, model=model, min_length=min_length, max_length=max_length, truncation=truncation)
      # Perform summarization operation on the document
      return summarizer.operation_on_document(df=self.df, name=featurename)
    except Exception as e:
      # Return exception if an error occurs
      return e

# Define a class for sentiment classification
class SentimentClassification:

  # Constructor method
  def __init__(self, dataset_name=None, version=None, split=None, nrows=None):
    # Retrieve dataset and display it using display_dataframe function
    self.df = display_dataframe(dataset=get_dataset(datasetname=dataset_name, version=version), name=split, nrows=nrows)

  # Method to perform sentiment analysis on text
  def text_sentiment(self, task="text-classification", model=None, min_length=None, max_length=None, truncation=None, featurename=None):
    try:
      # Initialize a sentiment analysis object using LLMTask class
      sentiment = LLMTask(task=task, model=model, min_length=min_length, max_length=max_length, truncation=truncation)
      # Perform sentiment analysis on the document
      return sentiment.operation_on_document(df=self.df, name=featurename)
    except Exception as e:
      # Return exception if an error occurs
      return e


In [7]:
# Create an instance of the Summarization class with specified dataset parameters
summarizer = Summarization(dataset_name="xsum", version='1.2.0', split='train', nrows=10)

# Call the summarize_text method of the summarizer object
# Pass model=t5-small, min_length=20, max_length=40, truncation=True, and featurename='document'
# Store the result in the variable result
result = summarizer.summarize_text(model="t5-small", min_length=20, max_length=40, truncation=True, featurename='document')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for xsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/xsum
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [8]:
result

[{'summary_text': 'the full cost of damage in Newton Stewart is still being assessed . many roads in peeblesshire remain badly affected by standing water . a flood alert remains in place across the'},
 {'summary_text': 'a fire alarm went off at the Holiday Inn in Hope Street on Saturday . guests were asked to leave the hotel . the two buses were parked side-by-side in'},
 {'summary_text': 'Sebastian Vettel will start third ahead of team-mate Kimi Raikkonen . stewards only handed Hamilton a reprimand after governing body said "n'},
 {'summary_text': 'the 67-year-old is accused of committing the offences between March 1972 and October 1989 . he denies all the charges, including two counts of indecency'},
 {'summary_text': 'a man receiving psychiatric treatment at the clinic threatened to shoot himself and others . the incident comes amid tension in Istanbul following several attacks in crowded areas .'},
 {'summary_text': 'Gregor Townsend gave a debut to powerhouse wing Taqele Naiyaravor

In [ ]:
# Create an instance of the SentimentClassification class with specified dataset parameters
sentiment_classification = SentimentClassification(dataset_name="poem_sentiment", split='train', nrows=10)

# Call the text_sentiment method of the sentiment_classification object
# Pass model="nickwong64/bert-base-uncased-poems-sentiment" and featurename='verse_text'
# Store the result in the variable result
result = sentiment_classification.text_sentiment(model="nickwong64/bert-base-uncased-poems-sentiment", featurename='verse_text')


Generating train split:   0%|          | 0/892 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/105 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/104 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/923 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/348 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
result = pd.DataFrame(result)

In [ ]:
result

,label,score
0,positive,0.996594
1,no_impact,0.998741
2,negative,0.995966
3,mixed,0.968735
4,mixed,0.975967
5,mixed,0.966580
6,no_impact,0.998639
7,no_impact,0.998611
8,negative,0.996557
9,no_impact,0.998519


In [ ]:
# Define a class for zero-shot classification
class ZeroShotClassification(Pipelines):

    # Constructor method
    def __init__(self, task="zero-shot-classification", model=None, min_length=None, max_length=None, truncation=None):
        # Call the constructor of the parent class using super()
        super().__init__(task=task, model=model, min_length=min_length, max_length=max_length, truncation=truncation)

    # Method for categorizing articles into predefined labels
    def categorize_article(self, labels=[], article=None):
        """
        This helper function defines the categories (labels) which the model must use to label articles.
        Note that our model was NOT fine-tuned to use these specific labels,
        but it "knows" what the labels mean from its more general training.

        This function then prints out the predicted labels alongside their confidence scores.
        """
        # Call the pipeline_obj method of the parent class, passing the article and labels
        results = self.pipeline_obj(article, labels)
        # Remove the "sequence" key from the results
        del results["sequence"]
        # Convert the results to a pandas DataFrame and return it
        return pd.DataFrame(results)





/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
user_input = "NC Chief Farooq Abdullah while talking about the Lok Sabha elections said, PM Modi has right to give statements. It is up to you to decide what is right and what is wrong. The price of edible oil has skyrocketed, poverty has surged. People are struggling with electricity bills. Youths are not getting jobs. You should ask people who are suffering. Medicine prices have swelled. They talk Ram Mandir. Have they made it? Everybody donated. It is our mistake that we have kept Quran but have not read it."

# Predefined candidate labels
candidate_labels = ["politics", "finance", "sports", "science and technology", "pop culture", "breaking news"]

# Initialize a ZeroShotClassification object with a specific model and minimum length
zero = ZeroShotClassification(model='cross-encoder/nli-deberta-v3-small', min_length=40)

# Call the categorize_article method to categorize the user input article
# Pass the predefined candidate labels and the user input article
zero_shot_df = zero.categorize_article(labels=candidate_labels, article=user_input)


/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
zero_shot_df

,labels,scores
0,politics,0.253064
1,finance,0.215408
2,breaking news,0.194973
3,science and technology,0.175497
4,pop culture,0.106231
5,sports,0.054826


In [ ]:
from transformers import pipeline, AutoTokenizer
import torch

model = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)